In [49]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_excel('../../data/Donn_es_Assurance_S2.1.xlsx',sheet_name='personne_morale')

dataframe shape

In [3]:
df.shape

(747, 8)

checking for duplicates

In [4]:
df.duplicated().sum()

np.int64(0)

checking for missing values

In [5]:
df.isna().sum()

REF_PERSONNE              0
RAISON_SOCIALE            0
MATRICULE_FISCALE         0
LIB_SECTEUR_ACTIVITE      0
LIB_ACTIVITE              8
VILLE                   427
LIB_GOUVERNORAT         481
VILLE_GOUVERNORAT         0
dtype: int64

Missing values in VILLE and LIB_GOUVERNORAT generally does not affect our recommendation system.

In [6]:
df.nunique()

REF_PERSONNE            747
RAISON_SOCIALE          747
MATRICULE_FISCALE       747
LIB_SECTEUR_ACTIVITE     61
LIB_ACTIVITE            251
VILLE                   159
LIB_GOUVERNORAT          19
VILLE_GOUVERNORAT       167
dtype: int64

* REF_PERSONNE, RAISON_SOCIALE, MATRICULE_FISCALE can be considered as unique identifiers
* we have 61 unique LIB_SECTEUR_ACTIVITE and 251 LIB_ACTIVITE (we need to check if some categories are well encoded)


Columns dtypes

In [7]:
df.dtypes

REF_PERSONNE             int64
RAISON_SOCIALE          object
MATRICULE_FISCALE       object
LIB_SECTEUR_ACTIVITE    object
LIB_ACTIVITE            object
VILLE                   object
LIB_GOUVERNORAT         object
VILLE_GOUVERNORAT       object
dtype: object

All columns are strings except REF_PERSONNE which is numeric

Columns Analysis:
* personne_morale: represents legal entities or companies
* REF_PERSONNE: id/reference code for each company.
* RAISON_SOCIALE: the official registered name of the company (This is the legal name under which the company operates)
* MATRICULE_FISCALE: tax identification number or fiscal registration number assigned to the company by the tax authority. (used for tax reporting and official financial transactions)
* LIB_SECTEUR_ACTIVITE: the sector of activity in which the company operates
* LIB_ACTIVITE: a more detailed description of the company's specific activity or business.
* VILLE: The city where the company is located or registered
* LIB_GOUVERNORAT: The governorate where the company is located
* VILLE_GOUVERNORAT: This represents the city within the governorate.

We can deal with the 8 missing values in LIB_ACTIVITE with LIB_SECTEUR_ACTIVITE to preserve meaningful information.

In [9]:
df[df['LIB_ACTIVITE'].isnull()]

,REF_PERSONNE,RAISON_SOCIALE,MATRICULE_FISCALE,LIB_SECTEUR_ACTIVITE,LIB_ACTIVITE,VILLE,LIB_GOUVERNORAT,VILLE_GOUVERNORAT
89,98018,Societe_000090,0000090G,INDUSTRIE,NaN,NaN,NaN,-
150,395189,Societe_000151,0000151R,AUCUN,NaN,NaN,NaN,-
213,550886,Societe_000214,0000214K,ACTIVITE SPORTIVE,NaN,NaN,NaN,-
222,553109,Societe_000223,0000223B,ACTIVITE SPORTIVE,NaN,NaN,NaN,-
225,557042,Societe_000226,0000226I,ÉDUCATION,NaN,NaN,NaN,-
227,557111,Societe_000228,0000228X,ACTIVITE SPORTIVE,NaN,NaN,NaN,-
489,557149,Societe_000490,0000490Y,AUCUN,NaN,Haffouz,ARIANA,Haffouz - ARIANA
679,557052,Societe_000680,0000680U,POSTES ET TÉLÉCOMMUNICATIONS,NaN,Berge Du Lac,TUNIS,Berge Du Lac - TUNIS


In [10]:
df['LIB_ACTIVITE'].fillna(df['LIB_SECTEUR_ACTIVITE'],inplace=True)

In [12]:
df.nunique()

REF_PERSONNE            747
RAISON_SOCIALE          747
MATRICULE_FISCALE       747
LIB_SECTEUR_ACTIVITE     61
LIB_ACTIVITE            256
VILLE                   159
LIB_GOUVERNORAT          19
VILLE_GOUVERNORAT       167
dtype: int64

EDA for activity/sector columns

Questions:
* How many unique sectors and activities exist?
* Are these sectors with very few activities(or vice versa)?

In [13]:
print('Unique Sectors:',df['LIB_SECTEUR_ACTIVITE'].nunique())
print('Unique Actvities:',df['LIB_ACTIVITE'].nunique())

Unique Sectors: 61
Unique Actvities: 256


In [17]:
# Frequency distribution
print('\nTop 10 sectors:')
print(df['LIB_SECTEUR_ACTIVITE'].value_counts().head(10))

print('\nTop 10 Activities:')
print(df['LIB_ACTIVITE'].value_counts().head(10))


Top 10 sectors:
LIB_SECTEUR_ACTIVITE
AUCUN                                              93
COMMERCE DE GROS ET INTERMÉDIAIRES DU COMMERCE     51
ACTIVITES IARD TARIFIABLES                         50
SERVICES FOURNIS PRINCIPALEMENT AUX ENTREPRISES    41
CONSTRUCTION                                       38
ACTIVITÉS INFORMATIQUES                            29
ACTIVITÉS IMMOBILIÈRES                             29
HÔTELS ET RESTAURANTS                              29
AGRICULTURE, CHASSE, SERVICES ANNEXES              28
SANTÉ ET ACTION SOCIALE                            25
Name: count, dtype: int64

Top 10 Activities:
LIB_ACTIVITE
NON DEFINI                                      91
PROMOTION IMMOBILIÈRE DE LOGEMENTS              24
AUXILIAIRES D ASSURANCE                         16
CULTURE DE CÉRÉALES ; CULTURES INDUSTRIELLES    12
AGENCES ET BUREAUX                              12
BANQUES                                         12
CONSEIL POUR LES AFFAIRES ET LA GESTION         12
AUTRES

As we can see, we have many companies whose Sector and activities are not defined, since we don't know any information about them we can't recommend them anything, they will be filtered.

In [18]:
filtered_df = df[
    (df['LIB_SECTEUR_ACTIVITE']!='AUCUN') &
    (df['LIB_ACTIVITE']!='NON DEFINI')
]
print('original shape',df.shape)
print('Filtered shape:',filtered_df.shape)

original shape (747, 8)
Filtered shape: (654, 8)


In [ ]:
# Frequency distribution of filtered_df
print('\nTop 10 sectors:')
print(filtered_df['LIB_SECTEUR_ACTIVITE'].value_counts().head(10))

print('\nTop 10 Activities:')
print(filtered_df['LIB_ACTIVITE'].value_counts().head(10))


Top 10 sectors:
LIB_SECTEUR_ACTIVITE
COMMERCE DE GROS ET INTERMÉDIAIRES DU COMMERCE     51
ACTIVITES IARD TARIFIABLES                         50
SERVICES FOURNIS PRINCIPALEMENT AUX ENTREPRISES    41
CONSTRUCTION                                       38
ACTIVITÉS INFORMATIQUES                            29
ACTIVITÉS IMMOBILIÈRES                             29
HÔTELS ET RESTAURANTS                              29
AGRICULTURE, CHASSE, SERVICES ANNEXES              28
SANTÉ ET ACTION SOCIALE                            25
TRANSPORTS TERRESTRES                              23
Name: count, dtype: int64

Top 10 Activities:
LIB_ACTIVITE
PROMOTION IMMOBILIÈRE DE LOGEMENTS              24
AUXILIAIRES D ASSURANCE                         16
AGENCES ET BUREAUX                              12
BANQUES                                         12
CONSEIL POUR LES AFFAIRES ET LA GESTION         12
CULTURE DE CÉRÉALES ; CULTURES INDUSTRIELLES    12
AUTRES ACTIVITÉS SPORTIVES                      11
CONSTR

After filtering these rows, we can see that we don't have any undefined values in both columns, which means that rows which had undefined activities also had undefined sectors.

But we have some rows with missing Secteur and activite (decoded as empty string) we need to deal with them.We can replace them with activities

In [42]:
empty_mask = (
    filtered_df['LIB_SECTEUR_ACTIVITE'].str.strip().eq('') 
)
print(f"Number of empty string decoded sectors: {empty_mask.sum()}")
print('Sample rows with empty sectors:')
print(filtered_df[empty_mask])

Number of empty string decoded sectors: 0
Sample rows with empty sectors:
Empty DataFrame
Columns: [REF_PERSONNE, RAISON_SOCIALE, MATRICULE_FISCALE, LIB_SECTEUR_ACTIVITE, LIB_ACTIVITE, VILLE, LIB_GOUVERNORAT, VILLE_GOUVERNORAT]
Index: []


In [41]:
filtered_df['LIB_SECTEUR_ACTIVITE'] = filtered_df.apply(
    lambda row: row['LIB_ACTIVITE'] if empty_mask[row.name] and pd.notna(row['LIB_ACTIVITE']) else row['LIB_SECTEUR_ACTIVITE'],
    axis=1
)

we also need to apply the empty mask to LIB_ACTIVITE if there are empty string decoded values.

In [45]:
empty_mask_2 = (
    filtered_df['LIB_ACTIVITE'].str.strip().eq('') 
)
print(f"Number of empty string decoded sectors: {empty_mask_2.sum()}")
print('Sample rows with empty sectors:')
print(filtered_df[empty_mask_2])

Number of empty string decoded sectors: 0
Sample rows with empty sectors:
Empty DataFrame
Columns: [REF_PERSONNE, RAISON_SOCIALE, MATRICULE_FISCALE, LIB_SECTEUR_ACTIVITE, LIB_ACTIVITE, VILLE, LIB_GOUVERNORAT, VILLE_GOUVERNORAT]
Index: []


We can now map activities to sector, to check for any illogical mapping.

In [43]:
sector_activity_map = filtered_df.groupby('LIB_SECTEUR_ACTIVITE')['LIB_ACTIVITE'].unique().reset_index()
for _, row in sector_activity_map.head().iterrows():
    print(f"Sector:'{row['LIB_SECTEUR_ACTIVITE']}")
    print(f"Activities:{', '.join(row['LIB_ACTIVITE'])}\n")

Sector:'ACTIVITE SPORTIVE
Activities:ACTIVITE SPORTIVE

Sector:'ACTIVITES IARD TARIFIABLES
Activities:QUINCAILLERIES (COMMERCE), ELECTRICITE (COMMERCE D ARTICLES D ECLAIRAGE), HYGIENE (ETABLISSEMENT DE BAIN DOUCHE HAMMAM SAUNA), IMPRIMERIE, TYPOGRAPHIES ET OFFSET, AGENCE ( DE VOYAGE,ASSURANCE,IMMOBILIERE), ECOLES, COLLEGES, CENTRE DE FORMATION, SALLE DE SPECTACLE OU DE FETE, BOULANGERIE AVEC FOURS AU FUEL (SANS FABRICATION DE BISCUITS ET DE GATEAUX SECS), ELECTRICIENS (REPARATION ET VENTE DE FOURNITURES POUR INSTALLATIONS ELECTRIQUE A L EXEPTION DU MATERIEL TRAITE SOUS LA RUBRIQUE -RADIO TV- ), ABBATOIRS ET DEPENDANCES(AVEC CHAMBRE FRIGORIFIQUE D UNE CAPACITE < 120 M3), VERRERIE, VAISSELLES (MAGASIN DE VENTE), PHARMACIENS (VENTE EN DETAIL), BATIMENTS : ENTREPRENEURS DE MACONNERIE ET DE CONSTRUCTION, ALIMENTATION GENERALE (COMMERCE), LINGE DE MAISON (MAGASIN DE VENTE), MATIERES PLASTIQUES  (COMMERCE D ARTICLES EN), MEDECIN, RESTAURANT, SELF SERVICE ET CANTINE, OPTIQUE ACOUSTIQUE AVEC VE

In [47]:
filtered_df.isna().sum()

REF_PERSONNE              0
RAISON_SOCIALE            0
MATRICULE_FISCALE         0
LIB_SECTEUR_ACTIVITE      0
LIB_ACTIVITE              0
VILLE                   373
LIB_GOUVERNORAT         424
VILLE_GOUVERNORAT         0
dtype: int64